# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from pickle import PicklingError
from joblib import dump, load
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.metrics import make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, recall_score, classification_report, f1_score, accuracy_score, precision_score, confusion_matrix, matthews_corrcoef, roc_curve, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector as selector
from datetime import datetime
from tensorflow import keras
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.utils.data import evaluate_print
from IPython.display import display, Markdown
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Settings
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "device_role"
TIME_BUCKETS  = [
    "7d", 
    "2d", 
    "1d", 
    "12h",
    "6h", 
    "1h"
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def add_count_threat_role(x):
    threat = 1 if 1 in x.threat.values else 0
    return pd.Series([len(x), threat, x.role.iloc[0]], index=["count", "threat", "role"])


def train_classifier_on_single_users(df, classifier, time_bucket, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a dataframe where the results of the classifiers will be stored
        user_timings = pd.DataFrame()
        user_predictions = pd.DataFrame()

        for name, group in df.groupby("role"):

            # Clustering methods do not work if all counts are zero ("could not form valid cluster separation")
            if group["count"].sum() == 0:
                group["y_true"] = group["threat"]
                group["y_pred"] = 0
                group["scores"] = 0
                timings = pd.Series(
                    {
                        "training_time": 0,
                        "inference_time": 0
                    }
                )
                user_timings = user_timings.append(timings, ignore_index=True)
                user_predictions = pd.concat([user_predictions, group])
                continue

            # Create a clone of the classifier
            try:
                classifier = clone(classifier)
            except:
                pass
            
            # Update the parameters of the classifier according to the grid search
            classifier.set_params(**params)
            
            # Set the random state attribute of the classifier (if it has one)
            try:
                classifier.set_params(**{"random_state": randint(0, 2**32)})
            except Exception:
                pass

            # Define the transformers that do the rest of the preprocessing (scaling, encoding)
            numeric_transformer = Pipeline(steps=[
                ("scaler", RobustScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ("ohe", OneHotEncoder())
            ])


            # Create a pipeline that performs the feature selection and scaling
            pipe = Pipeline([
                ("column_transformer", ColumnTransformer(
                    transformers=[
                        ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                        ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                    ]
                )),
                ("classifier", classifier)
            ])
        
            # Benchmark the training
            start_training = timer()
            pipe.fit(group[["count"]])
            end_training = timer()

            # Benchmark the inference
            start_inference = timer()
            pipe.predict(group[["count"]])
            end_inference = timer()

            # Add the predictions and anomaly scores to the group dataframe
            group["y_true"] = group["threat"]
            group["y_pred"] = pipe.named_steps["classifier"].labels_
            group["scores"] = pipe.named_steps["classifier"].decision_scores_

            # Create a new series with the training metrics for the user iteration
            timings = pd.Series(
                {
                    "training_time": end_training - start_training,
                    "inference_time": end_inference - start_inference
                }
            )

            # Append the series to our user summary dataframes
            user_timings = user_timings.append(timings, ignore_index=True)
            user_predictions = pd.concat([user_predictions, group])

        # If there are inf values in the anomaly scores, replace it with the elsewise highest value
        max_without_infs = user_predictions["scores"].replace([np.inf, -np.inf], np.nan).max()
        user_predictions["scores"].replace(np.inf, max_without_infs, inplace=True)
        user_predictions["scores"].replace(np.nan, 0, inplace=True)

        # Find the right threshold to satisfy the contamination and add the final predictions
        upper = max_without_infs
        lower = 0
        for i in range(0,500):
            threshold = (upper + lower) / 2
            test = (user_predictions.scores > threshold).astype(bool)
            if test.sum() > CONTAMINATION * len(df):
                lower = threshold
            else:
                upper = threshold
        user_predictions["y_pred"] = test

        # After the predictions were added, calculate the evaluation metrics
        recall = recall_score(user_predictions.threat, user_predictions.y_pred)
        precision = precision_score(user_predictions.threat, user_predictions.y_pred)
        f1 = f1_score(user_predictions.threat, user_predictions.y_pred)
        cm = confusion_matrix(user_predictions.threat, user_predictions.y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(user_predictions.threat, user_predictions.scores)
            p_auc_10 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.3)
        except ValueError as e:
            print(e)

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": user_timings.training_time.sum(),
                "inference_time": user_timings.inference_time.sum(),
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": user_predictions.y_true,
                "y_pred": user_predictions.y_pred,
                "scores": user_predictions.scores,
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, time_bucket, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.time_bucket = time_bucket
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=1
                    )
            except (ValueError, IndexError) as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, time_bucket in enumerate(TIME_BUCKETS, start=1):

        display(Markdown(f"# {i}/{len(TIME_BUCKETS)} - Bucket: {time_bucket}"))

        # Group the dataframe by users and time buckets
        tmp = df.groupby([df["user"], pd.Grouper(freq=time_bucket, key="date")]).apply(add_count_threat_role)
        tmp = tmp.reindex(pd.MultiIndex.from_product(tmp.index.levels))
        tmp = tmp.fillna(0)

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=tmp,
            classifier=classifier,
            time_bucket=time_bucket,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "device"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 24 different hyperparameter combinations
 Training Time: 0.5305    Inference Time: 0.1685    pAUC: 0.5085 ± 0.0041   Recall: 0.0391 ± 0.0132   TN: 18874.0   FP: 27.0    FN: 173.0   TP: 6.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.5243    Inference Time: 0.3660    pAUC: 0.5385 ± 0.0107   Recall: 0.1162 ± 0.0210   TN: 18789.0   FP: 112.0   FN: 153.0   TP: 26.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 6.7911    Inference Time: 1.1447    pAUC: 0.5332 ± 0.0050   Recall: 0.1106 ± 0.0096   TN: 18780.0   FP: 121.0   FN: 156.0   TP: 23.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 12.0968   Inference Time: 2.0919    pAUC: 0.5270 ± 0.0032   Recall: 0.0972 ± 0.0076   TN: 18777.0   FP: 124.0   FN: 160.0   TP: 19.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4850    Inference Time: 0.1540    pAUC: 0.5263 ± 0.0164   Recall: 0.0894 ± 0.0331   TN: 18837.0   FP: 64.0    FN: 152.0   TP: 27.0   P

# 2/6 - Bucket: 2d

Testing 24 different hyperparameter combinations
 Training Time: 0.5676    Inference Time: 0.1823    pAUC: 0.5041 ± 0.0105   Recall: 0.0423 ± 0.0208   TN: 65919.0   FP: 71.0    FN: 239.0   TP: 21.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.6582    Inference Time: 0.4399    pAUC: 0.5357 ± 0.0075   Recall: 0.1154 ± 0.0135   TN: 65794.0   FP: 196.0   FN: 223.0   TP: 37.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 8.0503    Inference Time: 1.5219    pAUC: 0.5362 ± 0.0040   Recall: 0.1177 ± 0.0075   TN: 65774.0   FP: 216.0   FN: 226.0   TP: 34.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 14.4557   Inference Time: 2.9134    pAUC: 0.5373 ± 0.0045   Recall: 0.1200 ± 0.0092   TN: 65818.0   FP: 172.0   FN: 226.0   TP: 34.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.6059    Inference Time: 0.1931    pAUC: 0.5116 ± 0.0070   Recall: 0.0631 ± 0.0110   TN: 65808.0   FP: 182.0   FN: 239.0   TP: 21.0   

# 3/6 - Bucket: 1d

...

Testing 24 different hyperparameter combinations
 Training Time: 0.6269    Inference Time: 0.2157    pAUC: 0.4917 ± 0.0041   Recall: 0.0297 ± 0.0081   TN: 264351.0   FP: 171.0   FN: 458.0   TP: 20.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 5.2244    Inference Time: 0.7285    pAUC: 0.5116 ± 0.0023   Recall: 0.0707 ± 0.0045   TN: 263992.0   FP: 530.0   FN: 442.0   TP: 36.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 11.5545   Inference Time: 2.9354    pAUC: 0.5140 ± 0.0001   Recall: 0.0753 ± 0.0000   TN: 263970.0   FP: 552.0   FN: 442.0   TP: 36.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 20.4323   Inference Time: 5.5513    pAUC: 0.5141 ± 0.0001   Recall: 0.0753 ± 0.0000   TN: 263970.0   FP: 552.0   FN: 442.0   TP: 36.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.6266    Inference Time: 0.2111    pAUC: 0.4974 ± 0.0074   Recall: 0.0418 ± 0.0142   TN: 264225.0   FP: 297.0   FN: 447.0   TP: 31

# 5/6 - Bucket: 6h

Testing 24 different hyperparameter combinations
 Training Time: 0.7192    Inference Time: 0.2574    pAUC: 0.4858 ± 0.0023   Recall: 0.0192 ± 0.0049   TN: 529065.0   FP: 129.0   FN: 526.0   TP: 15.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 6.0651    Inference Time: 1.1109    pAUC: 0.5048 ± 0.0025   Recall: 0.0577 ± 0.0050   TN: 528598.0   FP: 596.0   FN: 507.0   TP: 34.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 15.1545   Inference Time: 4.5057    pAUC: 0.5065 ± 0.0001   Recall: 0.0610 ± 0.0000   TN: 528550.0   FP: 644.0   FN: 508.0   TP: 33.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 28.0479   Inference Time: 8.9793    pAUC: 0.5065 ± 0.0001   Recall: 0.0610 ± 0.0000   TN: 528551.0   FP: 643.0   FN: 508.0   TP: 33.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.7201    Inference Time: 0.2544    pAUC: 0.4926 ± 0.0028   Recall: 0.0329 ± 0.0054   TN: 528702.0   FP: 492.0   FN: 518.0   TP: 23

# 6/6 - Bucket: 1h

Testing 24 different hyperparameter combinations
 Training Time: 1.6623    Inference Time: 0.6580    pAUC: 0.4775 ± 0.0010   Recall: 0.0047 ± 0.0017   TN: 3176248.0   FP: 543.0   FN: 1344.0   TP: 10.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 12.7014   Inference Time: 4.6191    pAUC: 0.4800 ± 0.0003   Recall: 0.0111 ± 0.0007   TN: 3175312.0   FP: 1479.0   FN: 1338.0   TP: 16.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 47.3368   Inference Time: 19.7948   pAUC: 0.4807 ± 0.0000   Recall: 0.0126 ± 0.0000   TN: 3175268.0   FP: 1523.0   FN: 1337.0   TP: 17.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 87.3114   Inference Time: 37.5781   pAUC: 0.4807 ± 0.0000   Recall: 0.0126 ± 0.0000   TN: 3175268.0   FP: 1523.0   FN: 1337.0   TP: 17.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 1.6231    Inference Time: 0.6412    pAUC: 0.4790 ± 0.0009   Recall: 0.0089 ± 0.0020   TN: 3175633.0   FP: 1158.0   FN: 1

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 28 different hyperparameter combinations
 Training Time: 0.4834    Inference Time: 0.1487    pAUC: 0.5323 ± 0.0000   Recall: 0.0615 ± 0.0000   TN: 18766    FP: 135     FN: 168   TP: 11    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.6849    Inference Time: 0.1920    pAUC: 0.5257 ± 0.0000   Recall: 0.0559 ± 0.0000   TN: 18600    FP: 301     FN: 169   TP: 10    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 0.8497    Inference Time: 0.2264    pAUC: 0.5249 ± 0.0000   Recall: 0.0447 ± 0.0000   TN: 18765    FP: 136     FN: 171   TP: 8     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 1.0473    Inference Time: 0.2347    pAUC: 0.5250 ± 0.0000   Recall: 0.0447 ± 0.0000   TN: 18767    FP: 134     FN: 171   TP: 8     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.5292    Inference Time: 0.1774    pAUC: 0.5490 ± 0.0000   Recall: 0.0670 ± 0.0000   TN: 18714    FP: 187     FN: 167   TP: 12    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 2/6 - Bucket: 2d

Testing 28 different hyperparameter combinations
 Training Time: 0.6042    Inference Time: 0.2235    pAUC: 0.6031 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65322    FP: 668     FN: 260   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.8678    Inference Time: 0.2852    pAUC: 0.6305 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65340    FP: 650     FN: 260   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 1.1137    Inference Time: 0.3524    pAUC: 0.6166 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65255    FP: 735     FN: 260   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 1.3755    Inference Time: 0.4256    pAUC: 0.5960 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65256    FP: 734     FN: 260   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 5.5638    Inference Time: 5.1585    pAUC: 0.6199 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65327    FP: 663     FN: 260   TP: 0     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 3/6 - Bucket: 1d

...

Testing 28 different hyperparameter combinations
 Training Time: 1.0088    Inference Time: 0.4362    pAUC: 0.6190 ± 0.0000   Recall: 0.0858 ± 0.0000   TN: 261903   FP: 2619    FN: 437   TP: 41    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 1.6507    Inference Time: 0.7381    pAUC: 0.6289 ± 0.0000   Recall: 0.1067 ± 0.0000   TN: 261911   FP: 2611    FN: 427   TP: 51    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 2.9033    Inference Time: 1.2410    pAUC: 0.6226 ± 0.0000   Recall: 0.1004 ± 0.0000   TN: 261967   FP: 2555    FN: 430   TP: 48    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 2.9617    Inference Time: 1.2775    pAUC: 0.6262 ± 0.0000   Recall: 0.1088 ± 0.0000   TN: 261984   FP: 2538    FN: 426   TP: 52    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.0954    Inference Time: 0.4561    pAUC: 0.6260 ± 0.0000   Recall: 0.1402 ± 0.0000   TN: 261949   FP: 2573    FN: 411   TP: 67    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 5/6 - Bucket: 6h

Testing 28 different hyperparameter combinations
 Training Time: 1.5398    Inference Time: 0.7797    pAUC: 0.6537 ± 0.0000   Recall: 0.1091 ± 0.0000   TN: 523648   FP: 5546    FN: 482   TP: 59    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.5578    Inference Time: 1.2292    pAUC: 0.6582 ± 0.0000   Recall: 0.1312 ± 0.0000   TN: 523958   FP: 5236    FN: 470   TP: 71    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.7768    Inference Time: 1.9287    pAUC: 0.6699 ± 0.0000   Recall: 0.1386 ± 0.0000   TN: 524114   FP: 5080    FN: 466   TP: 75    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.7900    Inference Time: 2.3309    pAUC: 0.6792 ± 0.0000   Recall: 0.1386 ± 0.0000   TN: 524137   FP: 5057    FN: 466   TP: 75    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.4872    Inference Time: 0.7077    pAUC: 0.7012 ± 0.0000   Recall: 0.1386 ± 0.0000   TN: 523797   FP: 5397    FN: 466   TP: 75    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 6/6 - Bucket: 1h

Testing 28 different hyperparameter combinations
 Training Time: 5.7681    Inference Time: 3.6771    pAUC: 0.8474 ± 0.0000   Recall: 0.2378 ± 0.0000   TN: 3144400   FP: 32391   FN: 1032   TP: 322   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 10.8470   Inference Time: 6.8350    pAUC: 0.8499 ± 0.0000   Recall: 0.1588 ± 0.0000   TN: 3151426   FP: 25365   FN: 1139   TP: 215   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 16.6289   Inference Time: 10.4616   pAUC: 0.8518 ± 0.0000   Recall: 0.2563 ± 0.0000   TN: 3139971   FP: 36820   FN: 1007   TP: 347   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 20.7854   Inference Time: 12.9951   pAUC: 0.8431 ± 0.0000   Recall: 0.1396 ± 0.0000   TN: 3154819   FP: 21972   FN: 1165   TP: 189   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 5.4550    Inference Time: 3.5297    pAUC: 0.8575 ± 0.0000   Recall: 0.2474 ± 0.0000   TN: 3147847   FP: 28944   FN: 1019   TP: 335   Params: {'n_bins': 8, 'n_random_c

# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 0.5188    Inference Time: 0.2269    pAUC: 0.5567 ± 0.0000   Recall: 0.0782 ± 0.0000   TN: 18722    FP: 179     FN: 165   TP: 14    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5567 ± 0.0000
Recall: 0.0782 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 0.3503    Inference Time: 0.1696    pAUC: 0.5951 ± 0.0000   Recall: 0.1423 ± 0.0000   TN: 65353    FP: 637     FN: 223   TP: 37    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5951 ± 0.0000
Recall: 0.1423 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 0.4445    Inference Time: 0.2792    pAUC: 0.6324 ± 0.0000   Recall: 0.1130 ± 0.0000   TN: 261873   FP: 2649    FN: 424   TP: 54    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6324 ± 0.0000
Recall: 0.1130 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 0.9243    Inference Time: 0.4103    pAUC: 0.7131 ± 0.0000   Recall: 0.1516 ± 0.0000   TN: 523993   FP: 5201    FN: 459   TP: 82    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7131 ± 0.0000
Recall: 0.1516 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 1.5458    Inference Time: 2.0625    pAUC: 0.8469 ± 0.0000   Recall: 0.1920 ± 0.0000   TN: 3147409   FP: 29382   FN: 1094   TP: 260   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8469 ± 0.0000
Recall: 0.1920 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 0.4327    Inference Time: 0.1923    pAUC: 0.5846 ± 0.0000   Recall: 0.1229 ± 0.0000   TN: 18733    FP: 168     FN: 157   TP: 22    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5846 ± 0.0000
Recall: 0.1229 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 0.3419    Inference Time: 0.1612    pAUC: 0.5990 ± 0.0000   Recall: 0.1731 ± 0.0000   TN: 65322    FP: 668     FN: 215   TP: 45    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5990 ± 0.0000
Recall: 0.1731 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 0.4233    Inference Time: 0.2671    pAUC: 0.6105 ± 0.0000   Recall: 0.1130 ± 0.0000   TN: 261933   FP: 2589    FN: 424   TP: 54    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6105 ± 0.0000
Recall: 0.1130 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 0.5005    Inference Time: 0.4128    pAUC: 0.6974 ± 0.0000   Recall: 0.1331 ± 0.0000   TN: 523959   FP: 5235    FN: 469   TP: 72    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6974 ± 0.0000
Recall: 0.1331 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 1.3318    Inference Time: 1.9810    pAUC: 0.8361 ± 0.0000   Recall: 0.2090 ± 0.0000   TN: 3145816   FP: 30975   FN: 1071   TP: 283   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8361 ± 0.0000
Recall: 0.2090 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

import sklearn
# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [1, 2, 4],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 1.4371    Inference Time: 0.1882    pAUC: 0.5872 ± 0.0000   Recall: 0.1453 ± 0.0000   TN: 18759    FP: 142     FN: 153   TP: 26    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 0.7840    Inference Time: 0.1569    pAUC: 0.4991 ± 0.0004   Recall: 0.0000 ± 0.0000   TN: 18708    FP: 193     FN: 179   TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}: n_samples=2 should be >= n_clusters=4.
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weig

# 2/6 - Bucket: 2d

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 1.1327    Inference Time: 0.1933    pAUC: 0.6196 ± 0.0000   Recall: 0.1577 ± 0.0000   TN: 65317    FP: 673     FN: 219   TP: 41    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 0.7857    Inference Time: 0.1602    pAUC: 0.5921 ± 0.0002   Recall: 0.0154 ± 0.0000   TN: 65423    FP: 567     FN: 256   TP: 4     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 1.0153    Inference Time: 0.1656    pAUC: 0.5448 ± 0.0001   Recall: 0.0000 ± 0.0000   TN: 65378    FP: 612     FN: 260   TP: 0     Params: {'alpha': 0.2, 'bet

# 3/6 - Bucket: 1d

...

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 0.9963    Inference Time: 0.1943    pAUC: 0.5753 ± 0.0000   Recall: 0.0126 ± 0.0000   TN: 262881   FP: 1641    FN: 472   TP: 6     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 0.8691    Inference Time: 0.1699    pAUC: 0.6404 ± 0.0001   Recall: 0.1360 ± 0.0000   TN: 261982   FP: 2540    FN: 413   TP: 65    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 1.4855    Inference Time: 0.1859    pAUC: 0.6083 ± 0.0017   Recall: 0.0460 ± 0.0000   TN: 261807   FP: 2715    FN: 456   TP: 22    Params: {'alpha': 0.2, 'bet

# 5/6 - Bucket: 6h

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 1.0519    Inference Time: 0.1776    pAUC: 0.6565 ± 0.0000   Recall: 0.0887 ± 0.0000   TN: 523863   FP: 5331    FN: 493   TP: 48    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 0.8759    Inference Time: 0.1647    pAUC: 0.7140 ± 0.0000   Recall: 0.1645 ± 0.0000   TN: 524023   FP: 5171    FN: 452   TP: 89    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 1.1845    Inference Time: 0.1723    pAUC: 0.6908 ± 0.0005   Recall: 0.0821 ± 0.0235   TN: 524680   FP: 4514    FN: 507   TP: 34    Params: {'alpha': 0.2, 'bet

# 6/6 - Bucket: 1h

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 1.1950    Inference Time: 0.1926    pAUC: 0.8381 ± 0.0001   Recall: 0.1182 ± 0.0000   TN: 3154575   FP: 22216   FN: 1194   TP: 160   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 1.1763    Inference Time: 0.1813    pAUC: 0.8528 ± 0.0001   Recall: 0.2004 ± 0.0009   TN: 3145520   FP: 31271   FN: 1082   TP: 272   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 2.1238    Inference Time: 0.1873    pAUC: 0.8274 ± 0.0051   Recall: 0.2019 ± 0.0101   TN: 3140927   FP: 35864   FN: 1074   TP: 280   Params: {'alpha': 0.2

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 168 different hyperparameter combinations
 Training Time: 0.3331    Inference Time: 0.1121    pAUC: 0.5587 ± 0.0000   Recall: 0.1117 ± 0.0000   TN: 18762    FP: 139     FN: 159   TP: 20    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3451    Inference Time: 0.1189    pAUC: 0.5587 ± 0.0000   Recall: 0.1117 ± 0.0000   TN: 18762    FP: 139     FN: 159   TP: 20    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3259    Inference Time: 0.1117    pAUC: 0.5587 ± 0.0000   Recall: 0.1117 ± 0.0000   TN: 18762    FP: 139     FN: 159   TP: 20    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3280    Inference Time: 0.1116    pAUC: 0.5587 ± 0.0000   Recall: 0.1117 ± 0.0000   TN: 18762    FP: 139     FN: 159   TP: 20    Params: {'n_components': 1, 'stan

# 2/6 - Bucket: 2d

Testing 168 different hyperparameter combinations
 Training Time: 0.3394    Inference Time: 0.1148    pAUC: 0.5687 ± 0.0000   Recall: 0.1231 ± 0.0000   TN: 65375    FP: 615     FN: 228   TP: 32    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3422    Inference Time: 0.1180    pAUC: 0.5687 ± 0.0000   Recall: 0.1231 ± 0.0000   TN: 65375    FP: 615     FN: 228   TP: 32    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3353    Inference Time: 0.1140    pAUC: 0.5687 ± 0.0000   Recall: 0.1231 ± 0.0000   TN: 65375    FP: 615     FN: 228   TP: 32    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3340    Inference Time: 0.1138    pAUC: 0.5687 ± 0.0000   Recall: 0.1231 ± 0.0000   TN: 65375    FP: 615     FN: 228   TP: 32    Params: {'n_components': 1, 'stan

# 3/6 - Bucket: 1d

...

Testing 168 different hyperparameter combinations
 Training Time: 0.3211    Inference Time: 0.1074    pAUC: 0.5548 ± 0.0000   Recall: 0.1046 ± 0.0000   TN: 262372   FP: 2150    FN: 428   TP: 50    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3157    Inference Time: 0.1055    pAUC: 0.5548 ± 0.0000   Recall: 0.1046 ± 0.0000   TN: 262372   FP: 2150    FN: 428   TP: 50    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3184    Inference Time: 0.1055    pAUC: 0.5548 ± 0.0000   Recall: 0.1046 ± 0.0000   TN: 262372   FP: 2150    FN: 428   TP: 50    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3223    Inference Time: 0.1070    pAUC: 0.5548 ± 0.0000   Recall: 0.1046 ± 0.0000   TN: 262372   FP: 2150    FN: 428   TP: 50    Params: {'n_components': 1, 'stan

# 5/6 - Bucket: 6h

Testing 168 different hyperparameter combinations
 Training Time: 0.3259    Inference Time: 0.1081    pAUC: 0.6038 ± 0.0000   Recall: 0.1091 ± 0.0000   TN: 523941   FP: 5253    FN: 482   TP: 59    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3510    Inference Time: 0.1153    pAUC: 0.6038 ± 0.0000   Recall: 0.1091 ± 0.0000   TN: 523941   FP: 5253    FN: 482   TP: 59    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3244    Inference Time: 0.1077    pAUC: 0.6038 ± 0.0000   Recall: 0.1091 ± 0.0000   TN: 523941   FP: 5253    FN: 482   TP: 59    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3260    Inference Time: 0.1073    pAUC: 0.6038 ± 0.0000   Recall: 0.1091 ± 0.0000   TN: 523941   FP: 5253    FN: 482   TP: 59    Params: {'n_components': 1, 'stan

# 6/6 - Bucket: 1h

Testing 168 different hyperparameter combinations
 Training Time: 0.3417    Inference Time: 0.1134    pAUC: 0.8169 ± 0.0000   Recall: 0.0805 ± 0.0000   TN: 3149814   FP: 26977   FN: 1245   TP: 109   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3420    Inference Time: 0.1152    pAUC: 0.8169 ± 0.0000   Recall: 0.0805 ± 0.0000   TN: 3149814   FP: 26977   FN: 1245   TP: 109   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3416    Inference Time: 0.1154    pAUC: 0.8169 ± 0.0000   Recall: 0.0805 ± 0.0000   TN: 3149814   FP: 26977   FN: 1245   TP: 109   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3392    Inference Time: 0.1133    pAUC: 0.8169 ± 0.0000   Recall: 0.0805 ± 0.0000   TN: 3149814   FP: 26977   FN: 1245   TP: 109   Params: {'n_components': 

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 107.0154   Inference Time: 2.1592    pAUC: 0.4921 ± 0.0001   Recall: 0.0000 ± 0.0000   TN: 18735    FP: 166     FN: 179   TP: 0     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 110.0976   Inference Time: 1.9829    pAUC: 0.4909 ± 0.0009   Recall: 0.0000 ± 0.0000   TN: 18705    FP: 196     FN: 179   TP: 0     Params: {'hidden_activation': 'sigmoid',

# 2/6 - Bucket: 2d

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 124.3475   Inference Time: 2.8340    pAUC: 0.5738 ± 0.0029   Recall: 0.0108 ± 0.0015   TN: 65205    FP: 785     FN: 257   TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features


# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 3/4 - Bucket: 1d

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 100.3558   Inference Time: 3.9512    pAUC: 0.6633 ± 0.0077   Recall: 0.1326 ± 0.0219   TN: 131076.0   FP: 1139.0   FN: 254.0   TP: 31.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_2" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_2/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_2/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • y=

# 4/4 - Bucket: 12h

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 119.8929   Inference Time: 5.3340    pAUC: 0.6250 ± 0.0138   Recall: 0.1226 ± 0.0125   TN: 261951   FP: 2571    FN: 413   TP: 65    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_1446" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_1446/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_1446/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  •

# 5/4 - Bucket: 6h

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 121.3715   Inference Time: 5.6307    pAUC: 0.6837 ± 0.0206   Recall: 0.1567 ± 0.0170   TN: 524074   FP: 5120    FN: 448   TP: 93    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_3594" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_3594/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_3594/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  •

# 6/4 - Bucket: 1h

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 141.0457   Inference Time: 9.3880    pAUC: 0.8415 ± 0.0095   Recall: 0.1674 ± 0.0188   TN: 3147929   FP: 28862   FN: 1087   TP: 267   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_5742" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_5742/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_5742/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
 